## Setup

Using [geemap docker](https://hub.docker.com/r/bkavlak/geemap)

```bash
# pull
docker pull bkavlak/geemap:latest

# check
docker image ls

# run
docker run -it --name geemap -p 8888:8888 -p 6006:6006 -v '/Users/bbest/My Drive/projects/offhab/data/_geemap':/geemap/data bkavlak/geemap:latest bash
# WARNING: The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested
```

### Shutdown VM

```bash
docker stop geemap
```

### Restart VM

```bash
docker start geemap
docker exec -it geemap bash

# inside VM
jupyter notebook --ip=0.0.0.0 --port=8888 --allow-root
# visit displayed URL, eg http://127.0.0.1:8888/?token=4e18b4332bbf0a44267f18e111687c141dc1f15ec191236e
```

# Initialize

In [ ]:
import ee

ee.Initialize()

In [ ]:
import geemap
import pandas as pd
import os
import time

dir_csv = '/geemap/data'

# get image of zones
im_zones_v1 = ee.Image('projects/ee-offhab/assets/im_zones_v1')

# get imagecollection of taxa layers
ic_spp = ee.ImageCollection('projects/ee-offhab/assets/lyrs_ds_aphia')

# get image_ids
im_sp_ids = ic_spp.aggregate_array("system:id").getInfo()

# Zonal stats per spp with `for` loop

In [ ]:
t0 = time.time()
# iterate over images
for i in range(0,20):  # i=1 
    im_sp_id = im_sp_ids[i]
    im_sp = ee.Image(im_sp_id)

    lyr_key = os.path.basename(im_sp_id)
    #print(lyr_key) # am_1006979

    means_csv = f'{dir_csv}/{lyr_key}.csv'
    print(f'{i}: {means_csv}')

    im_sp_z = im_sp.addBands(im_zones_v1)
    #print(img_z.bandNames().getInfo()) # ['b1', 'b1_1'] # TODO: rename band

    # Grouped a mean reducer: change of nightlights by land cover category.
    means = im_sp_z.reduceRegion(
       reducer = ee.Reducer.mean().group(
           groupField = 1,
           groupName  = 'zone_id'),
       geometry  = im_zones_v1.geometry(),
       scale     = 481,
       maxPixels = 2.2e8).getInfo()['groups']
    # {'groups': [{'zone_id': 1}, {'mean': 61, 'zone_id': 2}, {'mean': 40.145293103263604, 'zone_id': 3}, {'mean': 97.8158977279919, 'zone_id': 4}, {'zone_id': 5}, {'zone_id': 6}, {'zone_id': 7}, {'zone_id': 8}, {'zone_id': 9}, {'zone_id': 10}, {'zone_id': 11}]}

    df = pd.DataFrame.from_records(means)
    # print(df)
    df.to_csv(means_csv, index = False)
    
sec_elapsed = time.time() - t0
print(f'time elapsed: {sec_elapsed:.2f} sec')
#  0:5: 5.24 sec 
# 0:20: 41.17 sec


# Zonal stats per spp with `toBands()` & `FeatureCollection`

In [ ]:
t0 = time.time()
fc_zones = ee.FeatureCollection('projects/ee-offhab/assets/fc_zones')
fc_zones_v1 = fc_zones.filter(ee.Filter.eq('zn_vrsn', 1))

means_csv = f'{dir_csv}/sp_z_0to20.csv'
ic_lyrs = ee.ImageCollection([ee.Image(x) for x in im_sp_ids[0:20]])
im_lyrs = ic_lyrs.toBands()
#print(im_lyrs.bandNames().getInfo())
# ['am_100599_b1', 'am_1006979_b1', 'am_1007713_b1', 'am_100832_b1', 'am_100833_b1']

# print(ic_lyrs.aggregate_array("system:id").getInfo())

r = im_lyrs.reduceRegions(
    collection = fc_zones_v1.select('zone_ky'),
    reducer    = ee.Reducer.mean(),
    scale      = 481,
    crs        = 'EPSG:4326').select(
        propertySelectors = '.*', # ['zone_ky', 'am_100599_b1'],
        retainGeometry = False).getInfo()
r_df = pd.DataFrame.from_records([x['properties'] for x in r['features']])
#print(r_df)
r_df.to_csv(means_csv, index = False)

sec_elapsed = time.time() - t0
print(f'time elapsed: {sec_elapsed:.2f} sec')
#  0:5: 125.65 sec
# 0:20: 176.48 sec

## Extra Methods

In [ ]:
url = r.getDownloadURL(
  filetype  = 'CSV',
  selectors = ['zone_ky', 'am_100599_b1'],
  filename  = 'lyr_zones_means.csv')
print(url)

In [ ]:
task = ee.batch.Export.table.toDrive(
  collection  = r_f,
  description =  'r_f_2cols',
  fileFormat  =  'CSV',
  selectors   = ['zone_ky', 'am_100599_b1'])
task.start()

In [ ]:
# ImageCollection.map()
def get_sp_zonal_stats(im_sp):
    #lyr_key = im_sp.get('lyr_key').getInfo()
    #means_csv = f'{dir_csv}/{lyr_key}.csv'
    #print(f'{means_csv}')
    im_sp_z = im_sp.addBands(ee.Image('projects/ee-offhab/assets/im_zones_v1'))
    means = im_sp_z.reduceRegion(
       reducer = ee.Reducer.mean().group(
           groupField = 1,
           groupName  = 'zone_id'),
       #geometry  = im_sp.geometry(),
       scale     = 481,
       maxPixels = 2.2e8)
    return means

reduced = ic_lyrs.map(get_sp_zonal_stats).flatten()
#.getInfo()['groups']
# EEException: Collection.map: A mapped algorithm must return a Feature or Image.